In [386]:
import os
import sys
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import xml.etree.ElementTree as ET
import json
import matplotlib
import matplotlib.pyplot as plt
import skimage.draw
import random

import mrcnn.model as modellib
import mrcnn.visualize as visualize
import mrcnn.config as config
import mrcnn.utils as utils

import warnings
warnings.filterwarnings("ignore")

#import random
#import math

In [2]:
class FacesConfig(config.Config):
    NAME = "Faces"
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 4

In [3]:
config = FacesConfig()

In [4]:
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [401]:
dataset.images_data['images'][0]

{'id': 1, 'width': 1066, 'height': 1600, 'file_name': 'image_1.jpeg'}

In [417]:
class FacesDataset(utils.Dataset):
    """Dataset class for faces dataset"""
    
    def __init__(self, images_list, annotations_path, dataset_dir):
        super().__init__(self)
        
        # Add classes
        self.add_class('Faces', 1, 'face')
        self.add_class('Faces', 2, 'nose')
        self.add_class('Faces', 3, 'mouth')
        self.images_data = self.load_images_data(annotations_path)
        
        # Add images
        
        for i, filename in enumerate(images_list):
            img_id = self.images_data['annotations'][i]['image_id']
            img = self.images_data['images'][img_id-1]
        
            fp = os.path.join(dataset_path, filename)
            img_h = img['height']
            img_w = img['width']
            
            annot = self.images_data['annotations'][i]
        
            self.add_image('Faces', 
                           image_id=i, 
                           path=fp,
                           cat_id = annot['category_id'],
                           height=img_h,
                           width=img_w,
                           bbox=annot['bbox'],
                           annotations=annot['segmentation'])        
        
        """
        for i in range(len(self.images_data['annotations'])):
            img_id = self.images_data['annotations'][i]['image_id']
            img = self.images_data['images'][img_id-1]
            
            filename = img['file_name']
            fp = os.path.join(dataset_path, filename)
            
            img_h = img['height']
            img_w = img['width']
            
            annot = self.images_data['annotations'][i]
            
            self.add_image('Faces', 
                           image_id=annot['image_id'], 
                           path=fp,
                           cat_id = annot['category_id'],
                           height=img_h,
                           width=img_w,
                           bbox=annot['bbox'],
                           annotations=annot['segmentation'])
            """
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
    
    
    def load_images_data(self, filepath):
        """JSON format annotations"""
        with open (filepath) as file:
            info = json.load(file)
        
        return info
            
        
    def load_mask(self, image_id, n_polygons):
        assert image_id > 0, "image_id cannot be negative"
        
        info = self.image_info[image_id*n_polygons-2:image_id*n_polygons]
        h = info[0]['height']
        w = info[0]['width']
        
        class_ids = np.zeros((n_polygons,), dtype=np.int32)
        mask = np.zeros((h,w, n_polygons), dtype=np.uint8)
        for i, p in enumerate(info):
            class_ids[i] = p['cat_id']

            x_s = p['annotations'][0][::2]
            y_s = p['annotations'][0][1::2]
            rr, cc = skimage.draw.polygon(y_s, x_s)
    
            mask[rr,cc,i] = 1
        
        
        return mask, class_ids
    
    
    ## USE RESIZE IMAGE utils.Dataset.resize_image?? and resize_mask
    ## only work with masks for now, do not annotate further until first round of training and testing

In [418]:
annots_path = os.path.abspath('annots/polygons_20.json')

In [419]:
dataset_path = os.path.abspath('./images/labeled')

In [420]:
dataset = FacesDataset(annots_path, dataset_path)

TypeError: __init__() missing 1 required positional argument: 'dataset_dir'

In [421]:
images_list = os.listdir(dataset_path)
random.seed(42)
random.shuffle(images_list)
val_size = 5
images_val = images_list[:val_size]
images_train = images_list[val_size:]
print(len(images_train), len(images_val))

15 5


In [422]:
images_train

['image_3.jpeg',
 'image_5.jpeg',
 'image_8.jpeg',
 'image_15.jpeg',
 'image_20.jpeg',
 'image_7.jpeg',
 'image_19.jpeg',
 'image_10.jpeg',
 'image_2.jpeg',
 'image_11.jpeg',
 'image_6.jpeg',
 'image_16.jpeg',
 'image_17.jpeg',
 'image_1.jpeg',
 'image_12.jpeg']

In [394]:
dataset.prepare()

In [423]:
dataset_train = FacesDataset(images_train, annots_path, dataset_path)

In [424]:
dataset_train.image_info

[{'id': 0,
  'source': 'Faces',
  'path': 'E:\\PersonalProjects\\FaceDetect\\images\\labeled\\image_3.jpeg',
  'cat_id': 2,
  'height': 1600,
  'width': 1066,
  'bbox': [424.59322033898303,
   597.6406779661016,
   189.71186440677963,
   234.8813559322034],
  'annotations': [[487.8305084745763,
    612.0949152542373,
    482.4101694915254,
    626.5491525423729,
    478.7966101694915,
    646.4237288135593,
    467.9559322033898,
    671.7186440677966,
    458.9220338983051,
    695.206779661017,
    457.1152542372881,
    729.5355932203389,
    444.4677966101695,
    738.5694915254237,
    433.62711864406776,
    754.8305084745763,
    424.59322033898303,
    781.9322033898305,
    444.4677966101695,
    794.5796610169491,
    476.98983050847454,
    810.8406779661017,
    498.67118644067796,
    827.1016949152543,
    542.0338983050847,
    832.522033898305,
    574.5559322033898,
    807.2271186440678,
    594.4305084745763,
    794.5796610169491,
    614.3050847457627,
    776.5118

In [382]:
model = modellib.MaskRCNN(mode='training',
                             config=FacesConfig(),
                             model_dir=os.getcwd())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [383]:
model.load_weights(filepath='mask_rcnn_coco.h5', by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                            "mrcnn_bbox", "mrcnn_mask"])